In [1]:
import pandas as pd
import numpy as np
import locale

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# read csv data as dataframe
data = pd.read_csv('/home/iik/Desktop/tiria_waldprüfung/data/2021-07-20_21-57_results.csv', delimiter=';')

# rename variables
data = data.rename(columns={'Schueler-ID': 'child_id', 
                            'Gewaehlt1': 'choice_1',
                            'Gewaehlt2':'choice_2',
                            'GesamtRichtig':'Gesamt_Richtig',
                            'GesamtFalsch':'Gesamt_Falsch',
                            'GesamtLevel': 'Gesamt_Level'})

# delete the tutorial tasks from df (trial 1 & trial 2) as well as test trials (which is ID == 101 and ==202) as they should not be included in the analysis
data = data.loc[~((data['Aufgabe']=='Trial 1') | (data['Aufgabe']=='Trial 2'))]

# delete all test trials 
data = data.loc[~((data.ID ==101) | (data.ID ==202))]

# transform datatype from variable 'Aufgabe' to type integer
data.Aufgabe = data.Aufgabe.astype(int)

# Create Date time object
data.Start = pd.to_datetime(data.Start, format='%d.%m.%Y %H:%M:%S')
data.Ende = pd.to_datetime(data.Ende, format='%d.%m.%Y %H:%M:%S')

In [3]:
# Create a dict with dfs, each df holds data of a Subtest
dict_test_df={}
test_name = data.Test.unique()
for test in test_name:
    df_test = data.loc[data['Test'] == test]
    dict_test_df[test]= df_test
    
#dict_test_df.keys()

In [4]:
# select a test
test = dict_test_df.get(1)

In [5]:
# create a unique id for each child
test.insert(0, 'unique_id', test.ID.astype(str)+'_'+ test.child_id.astype(str))

In [6]:
child_ids = test.unique_id.unique()
child_ids = list(sorted(child_ids))

## Transform data of subtest

In [7]:
# loop all child ids and create for each child a list with all relevant data
list_all_children=[]

for id_ in child_ids:
    
    # create a df by selecting the relevant child by id number
    child = test.loc[test['unique_id'] == id_]
    
    # for each subtest correct for possible reruns by selecting only the relevant amount of entries, considering time of entry  
    child.sort_values(by=['Start'], ascending=True, inplace=True)
    
    if ((child.Test.iloc[0] == 1) | (child.Test.iloc[0] == 2)):
        if child.shape[0]> 10:
            redundant = child.shape[0]-10
            child = child.head(-redundant)
            child.sort_values(by=['Aufgabe'], ascending = True, inplace=True)
    
    if child.Test.iloc[0] == 3:
        if child.shape[0]> 50:
            redundant = child.shape[0]-50
            child = child.head(-redundant)
            child.sort_values(by=['Aufgabe'], ascending = True, inplace=True)

    if child.Test.iloc[0] == 4:
        if child.shape[0]> 30:
            redundant = child.shape[0]-30
            child = child.head(-redundant)
            child.sort_values(by=['Aufgabe'], ascending = True, inplace=True)
    
    # create empty list which will be filled with all data for that relevant child
    child_list = list()
    
    # add the unique id of the child
    child_list.append(id_)
    
    # add the child id
    child_list.append(child.child_id.unique().tolist()[0])

    # add level of subtest
    child_list.append(child.Level.unique().tolist()[0])
    
    # add level averaged of all subtests
    child_list.append(child.Gesamt_Level.unique().tolist()[0])
    
    # add the class level
    child_list.append(child.Stufe.unique().tolist()[0])

    # add name of teacher
    child_list.append(child.Lehrer.unique().tolist()[0])
    
    # add class is
    child_list.append(child.ID.unique().tolist()[0])
    
    # add class name
    child_list.append(child.Klasse.unique().tolist()[0])
        
    # add test number
    child_list.append(child.Test.unique().tolist()[0])
    
    # add time needed to take the test
    child_list.append(child.Dauer.unique().tolist()[0])
    

    # sort the table according to task number
    child.sort_values(by=['Aufgabe'])
    
    # add choice 1
    list_choice_1 = child.choice_1.tolist()
    for choice1 in list_choice_1:
        child_list.append(choice1) 
    
    # add choice_2, only relevant for subtest 1 and subtest 2
    if ((child.Test.iloc[0] == 1) | (child.Test.iloc[0] == 2)):
        list_choice_2 = child.choice_2.tolist()
        for choice2 in list_choice_2:
            child_list.append(choice2) 
    
    # add Ergebnis
    list_ergebnis = child.Ergebnis.tolist()
    for ergebnis in list_ergebnis:
        child_list.append(ergebnis)
            
    # add time needed to do the test
    # this is needed as soon as we measure the time per task. To date, we only measure time needed per test
#    list_dauer = child.Dauer.tolist()
#    for dauer in list_dauer:
#        empty.append(dauer)
    
    # add the child list to the list containing all children
    list_all_children.append(child_list)

## Create column names
As subtests differ in their amount of items and choices, the amount of columns and column names differ between the subtest.  
On 21.07.2021, amounts of items and choices are as follows:  
Test 1 = 10 items, 2 choices  
Test 2 = 10 items, 2 choices  
Test 3 = 50 items, 1 choice  
Test 4 = 30 items, 1 choice   

In [8]:
# Create column names which are equal in all subtest
column_names=['unique_id',
              'child_id', 
              'subtest_result',
              'end_result',
              'class_level', 
              'teacher',
              'class_id',
              'class_name',
              'test',
              'time'
]

# Create column names which differ between the tests due to the varying amount of items and choices
list_aufgabe = test.Aufgabe.unique().tolist()

variablen1_2 = ['choice1', 'choice2', 'ergebnis']
variablen3_4 =['choice1', 'ergebnis']

if ((test.Test.iloc[0] == 1) | (test.Test.iloc[0] == 2)):
    for variable in variablen1_2: 
        for i in range(1, len(list_aufgabe)+1):
            test_number= str(test.Test.unique()[0])
            item_number = str(i)

            column_name = test_number+'_'+item_number+'_'+variable
            column_names.append(column_name)
else:
    for variable in variablen3_4: 
        for i in range(1, len(list_aufgabe)+1):
            test_number= str(test.Test.unique()[0])
            item_number = str(i)

            column_name = test_number+'_'+item_number+'_'+variable
            column_names.append(column_name)    

## Create DF from transformed data and column names

In [9]:
# Create df with all data and relevant column names
data_transformed = pd.DataFrame(list_all_children,
               columns =column_names)

## Data cleaning  
The data is now in the correct form for analysis but is not yet cleaned. This is done in the following and includes: 
- handle missing data
- convert values as 'True', 'False', 'keine Eingabe' to 1, 0 and NaN respectively

In [10]:
# Create lists of all columns which names contain 'choice' or 'ergebnis'
choice_cols = [col for col in data_transformed.columns if 'choice' in col]
ergebnis_cols = [col for col in data_transformed.columns if 'ergebnis' in col]

# Adjust choices values; the chosen process depends on subtest (subtest 4 differs from subtest 1, 2, 3)
for col in choice_cols:
#    if data_transformed.test.iloc[0] == 4:
#        data_transformed[str(col)] = data_transformed[str(col)].astype(str)
#        data_transformed[str(col)] = data_transformed[str(col)].replace({'True':1, 'False':0, 'Keine Auswahl': np.nan})
#        data_transformed[str(col)] = data_transformed[str(col)].astype(float)
#    else:
    data_transformed[str(col)] = data_transformed[str(col)].replace({'Keine Auswahl': np.nan})

In [11]:
# Adjust ergebnis values
for col in ergebnis_cols:
    data_transformed[str(col)] = data_transformed[str(col)].astype(str)
    data_transformed[str(col)] = data_transformed[str(col)].replace({'Keine Auswahl': np.nan})

    #data_transformed[str(col)] = data_transformed[str(col)].replace({'True':1, 'False':0, 'Keine Auswahl': np.nan})
    #data_transformed[str(col)] = data_transformed[str(col)].astype(float)

In [12]:
# Rename df for readability
data_cleaned = data_transformed

In [13]:
# Create csv from transformed and cleaned df
table_out_test_class = data_cleaned.to_csv('/home/iik/Desktop/tiria_waldprüfung/data/test1.csv', index=False)
print('ok')

ok
